In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D,
                                     AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,
                                     BatchNormalization,Dropout)
from tensorflow.keras.models import Model, load_model
# from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow


%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)



from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import os
import tensorflow.keras.layers as tfl

In [2]:
# 1024

BATCH_SIZE = 16
pixes = 512
IMG_SIZE = (pixes, pixes)
directory = './train_images'
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                            #  image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                            #  image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

Found 872 files belonging to 5 classes.
Using 698 files for training.
Found 872 files belonging to 5 classes.
Using 174 files for validation.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [27]:
# 1024, 1369,
def classify_model(input_shape = (256, 256, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    # X = tf.keras.layers.Resizing(pixes,pixes)(X_input)
   
    X = layers.Rescaling(1./255)(X_input)
    
    X = Conv2D(filters = 16, kernel_size = 3, strides = (1, 1), padding='valid',activation='relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)

    # X_shortcut = X
    
    X = Conv2D(filters = 64, kernel_size = 3, strides = (1, 1), padding='same',activation='relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)


    X = Conv2D(filters = 64, kernel_size = 3, strides = (1, 1), padding='valid',activation='relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)
    X = Dropout(0.2)(X)
    
    # X = Add()([X_shortcut,X])
    # X = Activation('relu')(X)
    # X_shortcut = X
    
    X = Conv2D(filters = 32, kernel_size = 5, strides = (1, 1), padding='valid',activation='relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)

    
    X = Conv2D(filters = 16, kernel_size = 3, strides = (1, 1), padding='same',activation='relu')(X)
    X = MaxPooling2D((4, 4))(X)

    # X = Add()([X_shortcut,X])
    # X = Activation('relu')(X)
    # X_shortcut = X
    
    X = Conv2D(filters = 16, kernel_size = 3, padding='valid',activation='relu')(X)
    X = AveragePooling2D((2,2))(X)

    ### END CODE HERE

    # output layer
    X = Flatten()(X)
    X = Dropout(0.2)(X)
    X = Dense(64, activation='relu')(X)
    X = Dense(32, activation='relu')(X)
    X = Dropout(0.2)(X)
    X = Dense(16, activation='relu')(X)
    X = Dense(5, activation='linear', kernel_initializer = glorot_uniform(seed=0))(X)


    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [28]:
tf.keras.backend.set_learning_phase(True)

model = classify_model(classes = 5)
model.summary()

C:\Users\EMEKA ONWUEPE\anaconda3\envs\homl3\lib\site-packages\keras\backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 rescaling_5 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 252, 252, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 252, 252, 64)      9280      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 125, 125, 64)     0         
 g2D)                                                        

In [29]:
# d_1 = np.zeros((252, 252, 16))
# d_2 = np.zeros((252, 252, 64))

# d_3 = np.zeros((16, 64))

In [30]:
# np.matmul(d_2, d_3)

In [31]:
np.random.seed(1)
tf.random.set_seed(2)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [32]:
initial_epochs = 4
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=initial_epochs)

Epoch 1/4
44/44 [==============================] - 369s 7s/step - loss: 1.0559 - accuracy: 0.5573 - val_loss: 1.2479 - val_accuracy: 0.6034
Epoch 2/4
44/44 [==============================] - 298s 7s/step - loss: 1.0064 - accuracy: 0.6003 - val_loss: 1.1320 - val_accuracy: 0.6034
Epoch 3/4
44/44 [==============================] - 295s 7s/step - loss: 0.9541 - accuracy: 0.6189 - val_loss: 1.0179 - val_accuracy: 0.6034
Epoch 4/4
44/44 [==============================] - 310s 7s/step - loss: 0.9271 - accuracy: 0.5931 - val_loss: 1.0058 - val_accuracy: 0.6034


In [ ]:
# model.save('/content/drive/MyDrive/classify.h5',save_format='h5')

In [35]:
test_dataset = image_dataset_from_directory(directory)

Found 872 files belonging to 5 classes.


In [54]:
test_dataset.class_names

['distorted', 'good', 'not_clear', 'not_result', 'wrong_oreintation']

In [57]:
test_dataset.load('./good')

NotFoundError: NewRandomAccessFile failed to Create/Open: ./good\dataset_spec.pb : The system cannot find the path specified.
; No such process

In [37]:
len(test_dataset.file_paths)

872

In [38]:
predictions = model.predict(test_dataset)

28/28 [==============================] - 113s 4s/step


In [39]:
predictions

array([[-1.6597395 ,  0.96521956, -1.0076818 , -1.7701011 ,  0.6418342 ],
       [-2.0591779 ,  1.1569911 , -1.2181765 , -2.1864133 ,  0.8012126 ],
       [-1.9549001 ,  1.1191351 , -1.1848466 , -2.0780861 ,  0.7576032 ],
       ...,
       [-1.9455495 ,  1.1035136 , -1.1561215 , -2.0775    ,  0.76126736],
       [-1.7681541 ,  1.0382186 , -1.1046677 , -1.887521  ,  0.6757176 ],
       [-1.6000786 ,  0.94149196, -0.9853392 , -1.7094097 ,  0.6157208 ]],
      dtype=float32)

In [49]:
predictions.shape

(872, 5)

In [52]:
np.argmax(predictions,axis=-1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,